<a href="https://colab.research.google.com/github/mantoan-thi/Wire-harness-analysis/blob/main/An%C3%A1lise_TE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instação de biblioteca para trabalhar com Excel

In [ ]:
!pip install xlsxwriter

# Nova secção

In [139]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from math import factorial
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

Carregando dados

In [129]:
df_ckt = pd.read_excel('/content/Consulta BOM v14.xlsm',sheet_name='CIRCUITO')

Selecionando somentes colunas importantes

In [130]:
# Selecionando somentes colunas importantes
def selecionar_colunas(df_circuitos):
  if 'OT_DISCRIMINADA' in df_circuitos.columns:
    df_circuitos = df_ckt[['PROD_TERM', 'OT', 'OT_DISCRIMINADA', 'CIRC_ESQ','CIR_DIR', 'TIPO', 
                                'BITOLA', 'COR', 'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ','ACC_ESQ_1',
                                'COD_OP_01_DIR','TERMINAL_DIR','ACC_DIR_1','SOLDA']].copy()

    # Obtenha os nomes dos índices para os quais a coluna 'OT_DISCRIMINADA' tem o valor SIM

    index_ot_discriminada = df_circuitos[df_circuitos['OT_DISCRIMINADA']=='SIM'].index
    if index_ot_discriminada.shape[0] > 0: # Excluindo esses índices de linha de dataFrame
      df_circuitos.drop(index_ot_discriminada, inplace=True)
      df_circuitos.drop(columns=['OT_DISCRIMINADA'],axis=1, inplace=True)
    df_circuitos.fillna(0,inplace=True)

  df_circuitos['COD_OP_01_ESQ'] = df_circuitos['COD_OP_01_ESQ'].apply(lambda x: 0 if x != 'U' else 'U')
  df_circuitos['COD_OP_01_DIR'] = df_circuitos['COD_OP_01_DIR'].apply(lambda x: 0 if x != 'U' else 'U')
  df_circuitos.head()
  # Quantidade de circuitos por part number
  lista_pn = []
  pn_anterior = 0
  for pn in df_circuitos['PROD_TERM']:
    if pn_anterior != pn:
      pn_anterior = pn
      qtde = df_circuitos[df_circuitos['PROD_TERM']==pn].shape[0]
      lista_pn.append([pn,qtde])
  lista_pn =pd.DataFrame(lista_pn,columns=['Part_number','Qtde'])
  lista_pn.sort_values(by=['Qtde'],ascending=False,inplace=True)
  lista_pn.reset_index(inplace=True,drop=True)


  qtde_pn = lista_pn.shape[0]
  menor_pn = lista_pn['Part_number'][-1:].values[0]
  maior_pn = lista_pn['Part_number'][0]
  Describ = lista_pn.describe()
  return lista_pn,df_circuitos


Comparando

In [131]:
def analisar(list_comp,tol):
  contar1 = 0
  contar2 = 0
  list_comp['Delta'] = 0
  while contar1 < list_comp.shape[0]:
    contar2 = contar1+1
    while contar2 < list_comp.shape[0]:
      if list_comp['PROD_TERM'].iloc[contar1]!=list_comp['PROD_TERM'].iloc[contar2]:
        # Tipo+bit+cor
        tipoA = list_comp['TIPO'].iloc[contar1]+list_comp['BITOLA'].iloc[contar1]+list_comp['COR'].iloc[contar1]
        tipoB = list_comp['TIPO'].iloc[contar2]+list_comp['BITOLA'].iloc[contar2]+list_comp['COR'].iloc[contar2]
        TAE = str(list_comp['COD_OP_01_ESQ'].iloc[contar1])+str(list_comp['TERMINAL_ESQ'].iloc[contar1])+str(list_comp['ACC_ESQ_1'].iloc[contar1])
        TAD = str(list_comp['COD_OP_01_DIR'].iloc[contar1])+str(list_comp['TERMINAL_DIR'].iloc[contar1])+str(list_comp['ACC_DIR_1'].iloc[contar1])
        TBE = str(list_comp['COD_OP_01_ESQ'].iloc[contar2])+str(list_comp['TERMINAL_ESQ'].iloc[contar2])+str(list_comp['ACC_ESQ_1'].iloc[contar2])
        TBD = str(list_comp['COD_OP_01_DIR'].iloc[contar2])+str(list_comp['TERMINAL_DIR'].iloc[contar2])+str(list_comp['ACC_DIR_1'].iloc[contar2])
        COMPA = int(list_comp['COMP'].iloc[contar1])
        COMPB = int(list_comp['COMP'].iloc[contar2])
        dif = abs(COMPA-COMPB)
        JTA = str(list_comp['SOLDA'].iloc[contar1])[0]
        JTB = str(list_comp['SOLDA'].iloc[contar2])[0]
        if tipoA == tipoB:                            # Comparando Tipo,Bit e cor
          if COMPA == COMPB or dif < tol:              # Checando dimensional
            if TAE==TBE and TAD==TBD:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'
            if TAE==TBD and TAD==TBE:                  # Terminais
              list_comp['Delta'].iloc[contar1] ='Comum' 
              list_comp['Delta'].iloc[contar2] ='Comum'

      contar2 +=1
    contar1+=1
    contar2 = 0
    list_comp['Delta'].loc[list_comp['Delta']==0]='DIF'
    lista_analisada = list_comp.sort_values(by=['OT'],ascending=True,inplace=True)
  return list_comp

In [132]:
def contar_saidas(x):
  valor = 0
  for e in nova_lista3['Saida_esq']:
    if e == x:
      valor +=1
  for d in nova_lista3['Saida_dir']:
    if d == x:
      valor +=1
  return valor

In [133]:
def contar_lig(e,d):
  contador = 0
  valor = 0
  while contador < nova_lista3.shape[0]:
    if nova_lista3['Saida_esq'].loc[contador:contador].any() == e and nova_lista3['Saida_dir'].loc[contador:contador].any()==d:
      valor +=1
    if nova_lista3['Saida_esq'].loc[contador:contador].any() == d and nova_lista3['Saida_dir'].loc[contador:contador].any() == e:
      valor +=1
    contador +=1
  return valor

In [134]:
def perct(l,c):
  return'{:.0%}'.format(l/c)

In [140]:
def calculate_combinations(n, r):
    return factorial(n) // factorial(r) // factorial(n-r)

In [135]:
def processar(new_df_circuitos):
  # separa as saídas dos circuitos
  new_df_circuitos['Saida_esq'] = new_df_circuitos['CIRC_ESQ'].str[-3:]
  new_df_circuitos['Saida_dir'] = new_df_circuitos['CIR_DIR'].str[-3:]

  # Calcula a quantidade de saídas
  new_df_circuitos['qtde_sds_esq'] = new_df_circuitos['Saida_esq'].apply(contar_saidas)
  new_df_circuitos['qtde_sds_dir'] = new_df_circuitos['Saida_dir'].apply(contar_saidas)

  # Calcula a quantidade de ligações
  new_df_circuitos['qtde_lig_esq'] = new_df_circuitos.apply(lambda x: contar_lig(x['Saida_esq'],x['Saida_dir']), axis=1)
  new_df_circuitos['qtde_lig_dir'] = new_df_circuitos['qtde_lig_esq']

  # Calcula o percetual de ligações

  new_df_circuitos['Perct_lig_esq'] = new_df_circuitos.apply(lambda x: perct(x['qtde_lig_esq'],x['qtde_sds_esq']), axis=1)
  new_df_circuitos['Perct_lig_dir'] = new_df_circuitos.apply(lambda x: perct(x['qtde_lig_dir'],x['qtde_sds_dir']), axis=1)
  return new_df_circuitos

In [ ]:
# Calcular a quantidade de comparações
lista_pn,df = selecionar_colunas(df_circuitos)
contador1 =0
contador2 = 0
lista_de_comparacao = []
Total = 0

n_comb = 0
resultado = []
n = calculate_combinations(lista_pn.shape[0], 2)
print('Quantidade de pns:',lista_pn.shape[0])
print('Quantidade de comparações:',n,'\n\n')
while contador1 <= lista_pn.shape[0]:
  contador2=contador1+1
  while contador2 < lista_pn.shape[0]:

    pnA = lista_pn['Part_number'][contador1]
    pnB = lista_pn['Part_number'][contador2]
    print('Num: ',n_comb,' Comparativo: ',pnA,' vs ',pnB,'// % Concluídos: ','{:.2%}'.format(n_comb/n))

    list_comp = df[(df['PROD_TERM'] == pnA) | (df['PROD_TERM']==pnB)]
    nova_lista = analisar(list_comp,50)
    nova_lista['Total'] = 1
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==pnA)]='Adicionar'
    nova_lista['Delta'].loc[(nova_lista['Delta']=='DIF') & (nova_lista['PROD_TERM']==pnB)]='Excluir'

    nova_lista1 = nova_lista.copy()
    nova_lista2 = nova_lista.copy()
    nova_lista3 = pd.pivot_table(nova_lista1,index=['OT', 'CIRC_ESQ', 'CIR_DIR', 'TIPO', 'BITOLA', 'COR',
       'COMP', 'COD_OP_01_ESQ', 'TERMINAL_ESQ', 'ACC_ESQ_1', 'COD_OP_01_DIR',
       'TERMINAL_DIR', 'ACC_DIR_1', 'SOLDA','Delta'],values=["Total"],columns=['PROD_TERM'],aggfunc=[np.sum],fill_value=0,)
    
    nova_lista3.columns = [col[2] for col in nova_lista3.columns]
    nova_lista3.reset_index(inplace=True)

    resumo = pd.pivot_table(nova_lista2,index=['PROD_TERM','Delta'],values=["Total"],aggfunc=np.sum)

    resumo = pd.DataFrame(resumo)
    resumo.reset_index(inplace=True)

    comuns = 0
    adicionar = 0
    Excluir = 0
    contador = 0
    while contador < int(resumo.shape[0])-1:
      if resumo['Delta'][contador] == 'Comum':
        comuns = resumo['Total'][contador]
      if resumo['Delta'][contador] == 'Adicionar':
        adicionar = resumo['Total'][contador]
      if resumo['Delta'][contador] == 'Excluir':
        Excluir = resumo['Total'][contador]
      contador+=1
    #perc = '{:.2%}'.format(1-((adicionar+Excluir)/(comuns*2)))
    perc = float(1-((adicionar+Excluir)/(comuns*2)))



    nova_lista3[pnA] = nova_lista3[pnA].apply(lambda x: pnA if x == 1 else 0)
    nova_lista3[pnB] = nova_lista3[pnB].apply(lambda x: pnB if x == 1 else 0)
    nova_lista4 = processar(nova_lista3)
    cont = 0
    sesq_ant = 0
    sdir_ant = 0
    qtde_saidaA = 0
    qtde_saidaB = 0
    lista_de_saidasA = []
    lista_de_saidasB = []
    qtde_ckt_A = []
    qtde_ckt_B = []
    while cont < nova_lista4.shape[0]:
      if nova_lista4['Delta'][cont]!='Comum' and nova_lista4['Perct_lig_esq'][cont] == '100%':
        if nova_lista4[pnA][cont]!=0:
          if sesq_ant != nova_lista4['Saida_esq'][cont] and sdir_ant != nova_lista4['Saida_dir'][cont]:
            sesq_ant = nova_lista4['Saida_esq'][cont]
            sdir_ant = nova_lista4['Saida_dir'][cont]
            qtde_saidaA+=1
            lista_de_saidasA.append(nova_lista4['Saida_esq'][cont])
            qtde_ckt_A.append(nova_lista4['qtde_sds_esq'][cont])
        else:
          if sesq_ant != nova_lista4['Saida_esq'][cont] and sdir_ant != nova_lista4['Saida_dir'][cont]:
            sdir_ant = nova_lista4['Saida_dir'][cont]
            sesq_ant = nova_lista4['Saida_esq'][cont]
            qtde_saidaB+=1
            lista_de_saidasB.append(nova_lista4['Saida_esq'][cont])
            qtde_ckt_B.append(nova_lista4['qtde_sds_esq'][cont])

      if nova_lista4['Delta'][cont]!='Comum' and nova_lista4['Perct_lig_dir'][cont] == '100%':
        if nova_lista4[pnA][cont]!=0:
          if sdir_ant != nova_lista4['Saida_dir'][cont] and sesq_ant != nova_lista4['Saida_esq'][cont]:
            sesq_ant = nova_lista4['Saida_esq'][cont]
            sdir_ant = nova_lista4['Saida_dir'][cont]
            qtde_saidaA+=1
            lista_de_saidasA.append(nova_lista4['Saida_dir'][cont])
            qtde_ckt_A.append(nova_lista4['qtde_sds_dir'][cont])
        else:
          if sdir_ant != nova_lista4['Saida_dir'][cont] and sesq_ant != nova_lista4['Saida_esq'][cont]:
            sesq_ant = nova_lista4['Saida_esq'][cont]
            sdir_ant = nova_lista4['Saida_dir'][cont]
            qtde_saidaB+=1
            lista_de_saidasB.append(nova_lista4['Saida_dir'][cont])
            qtde_ckt_B.append(nova_lista4['qtde_sds_dir'][cont])
      cont+=1
    resultado.append([perc,pnA,qtde_saidaA,', '.join(lista_de_saidasA),qtde_ckt_A,pnB,qtde_saidaB,', '.join(lista_de_saidasB),qtde_ckt_B])
  #lista_de_comparacao.append(lista_pn['Part_number'][contador1]+' vs '+lista_pn['Part_number'][contador2])
    contador2+=1
    n_comb +=1
  contador1+=1
  contador2 = 0
  
#lista_de_comparacao = pd.DataFrame(lista_de_comparacao,columns=['Part Numbers'])

Quantidade de pns: 53
Quantidade de comparações: 1378 


Num:  0  Comparativo:  82115F5750                A20  vs  82115F5Q90                000 // % Concluídos:  0.00%
Num:  1  Comparativo:  82115F5750                A20  vs  82115F5Q80                A00 // % Concluídos:  0.07%


Apresentando Resultados

In [122]:
def risk(simil,A,B):
  if simil == 1:
    return 'Alto'
  else:
    if (A+B)==1:
      return 'Moderado'
    else:
      return 'Baixo'

In [123]:
def ordenar(x):
  if x == 'Alto':
    return 1
  else:
    if x == 'Moderado':
      return 2
    else:
      return 3

In [124]:
New_resultado = pd.DataFrame(resultado,columns=['Similaridade','Part_numberA','Núm_saídas_diferentesA','Saídas_exclusivasA','Núm_CktA','Part_numberB','Núm_saídas_diferentesB','Saídas_exclusivasB','Núm_CktB'])
New_resultado['Risk2'] = New_resultado['Núm_saídas_diferentesA']+New_resultado['Núm_saídas_diferentesB']
New_resultado['Risk'] = New_resultado.apply(lambda x: risk(x['Similaridade'],x['Núm_saídas_diferentesA'],x['Núm_saídas_diferentesB']),axis=1)
New_resultado['Risk3'] = New_resultado['Risk'].apply(lambda x: ordenar(x))
New_resultado.sort_values(by=['Risk3'],inplace=True)
New_resultado.drop(columns=['Risk2','Risk3'],inplace=True)
New_resultado['Similaridade'] = New_resultado['Similaridade'].apply(lambda x: '{:.2%}'.format(x))

In [125]:
New_resultado

,Similaridade,Part_numberA,Núm_saídas_diferentesA,Saídas_exclusivasA,Núm_CktA,Part_numberB,Núm_saídas_diferentesB,Saídas_exclusivasB,Núm_CktB,Risk
7,100.00%,821840A300 C00,0,,[],821840A30B C00,0,,[],Alto
0,97.14%,821840A310 C00,1,169,[2],821840A300 C00,0,,[],Moderado
25,92.59%,821840A350 C00,0,,[],821840A340 C00,1,168,[1],Moderado
24,92.59%,821840A330 C00,1,168,[1],821840A810 A00,0,,[],Moderado
22,98.33%,821840A330 C00,1,168,[1],821840A350 C00,0,,[],Moderado
20,87.04%,821840A320 C00,0,,[],821840A340 C00,1,168,[1],Moderado
18,91.38%,821840A320 C00,0,,[],821840A330 C00,1,168,[1],Moderado
17,85.19%,821840A30B C00,1,168,[1],821840A810 A00,0,,[],Moderado
15,89.66%,821840A30B C00,1,168,[1],821840A350 C00,0,,[],Moderado
12,85.19%,821840A300 C00,1,168,[1],821840A810 A00,0,,[],Moderado
